
## Importing Saved weights and files

In [2]:
!wget --header 'Host: doc-0s-c4-docs.googleusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Cookie: AUTH_hk31mtbc0rnrqkfur2kl4nou3khj2ng2_nonce=ehinnftsse4q6' --header 'Upgrade-Insecure-Requests: 1' 'https://doc-0s-c4-docs.googleusercontent.com/docs/securesc/q2bh4litpt8gnnf5551ajijntavlltei/5t8toahv2qp7ej4k82bes2u0am3ngs73/1607404200000/13975366803291835212/13975366803291835212/1hG98-xdrMNoxFDwvLLP6UX-ytFpWQTF3?e=download&authuser=4&nonce=ehinnftsse4q6&user=13975366803291835212&hash=k51d9cq7rrcuqi1r4pnqfu5sabggjmfn' --output-document 'files.zip'

--2020-12-08 05:11:19--  https://doc-0s-c4-docs.googleusercontent.com/docs/securesc/q2bh4litpt8gnnf5551ajijntavlltei/5t8toahv2qp7ej4k82bes2u0am3ngs73/1607404200000/13975366803291835212/13975366803291835212/1hG98-xdrMNoxFDwvLLP6UX-ytFpWQTF3?e=download&authuser=4&nonce=ehinnftsse4q6&user=13975366803291835212&hash=k51d9cq7rrcuqi1r4pnqfu5sabggjmfn
Resolving doc-0s-c4-docs.googleusercontent.com (doc-0s-c4-docs.googleusercontent.com)... 172.217.12.225, 2607:f8b0:4004:807::2001
Connecting to doc-0s-c4-docs.googleusercontent.com (doc-0s-c4-docs.googleusercontent.com)|172.217.12.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-zip-compressed]
Saving to: ‘files.zip’

files.zip               [ <=>                ]   1.88G  64.9MB/s    in 31s     

2020-12-08 05:11:51 (61.3 MB/s) - ‘files.zip’ saved [2021847964]



In [3]:
!unzip files

Archive:  files.zip
  inflating: custom.data             
  inflating: custom.names            
  inflating: densenet_stage1_all-0.959.hdf5  
  inflating: densenet_stage2_all-0.804.hdf5  
  inflating: densenet_stage3_all-0.696.hdf5  
  inflating: templates/app.html      
  inflating: templates/old.html      
  inflating: test.txt                
  inflating: train.txt               
  inflating: yolov3_custom.cfg       
  inflating: yolov3_custom_4000.weights  


## Importing libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
import random
from tqdm import tqdm
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential, Model, load_model
!pip install flask-ngrok
import io
from base64 import encodebytes
from PIL import Image
from flask import jsonify
import json

## Importing Saved models

In [5]:
Stage_1_model = load_model('densenet_stage1_all-0.959.hdf5')
Stage_2_model = load_model('densenet_stage2_all-0.804.hdf5')
Stage_3_model = load_model('densenet_stage3_all-0.696.hdf5')

## Importing Darknet git

In [6]:
!/usr/local/cuda/bin/nvcc --version
# Check if GPU is selected as hardware accelerator
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if not '/device:GPU:0' in device_name:
    print('\nERROR: GPU is not selected as hardware accelerator!')
else:
    print(device_name)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
/device:GPU:0


In [7]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!ls
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make
%cd /content/

Cloning into 'darknet'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 14583 (delta 6), reused 9 (delta 4), pack-reused 14562
Receiving objects: 100% (14583/14583), 13.25 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (9916/9916), done.
/content/darknet
3rdparty	DarknetConfig.cmake.in	include		       results
build		darknet_images.py	json_mjpeg_streams.sh  scripts
build.ps1	darknet.py		LICENSE		       src
build.sh	darknet_video.py	Makefile	       video_yolov3.sh
cfg		data			net_cam_v3.sh	       video_yolov4.sh
cmake		image_yolov3.sh		net_cam_v4.sh
CMakeLists.txt	image_yolov4.sh		README.md
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPEN

## Generating report for a given Image

In [41]:
#Creating a single function which predicts wether Car is damaged or not and localizing the damage, severity of damage for a single img
def report(img_path):
    report_pred = []
    #Creating Train and test txt files:
    test_file = open('/content/test.txt','w')
    test_file.write('/content/'+img_path+'\n')
    test_file.close()
    
    img = load_img('/content/'+img_path,target_size = (256,256))
    #Converting into array
    img_arr = img_to_array(img)
    img_arr = img_arr.reshape((1,) + img_arr.shape)
    
    #Checking if Damaged or not
    s1_pred = Stage_1_model.predict(img_arr)
    if s1_pred <=0.5:
        report_pred.append('Damaged')
    else:
        report_pred.append('Not Damaged')
        return report_pred
    #Checking for Damage Localization
    s2_pred = Stage_2_model.predict(img_arr)
    n = np.argmax(s2_pred)
    if n == 0:
        report_pred.append('Front')
    elif n ==1:
        report_pred.append('Rear')
    else:
        report_pred.append('Side')
    
    #Checking for Damage Severity
    s3_pred = Stage_3_model.predict(img_arr)
    c = np.argmax(s3_pred)
    if c == 0:
        report_pred.append('Minor')
    elif c ==1:
        report_pred.append('Moderate')
    elif c==2:
        report_pred.append('Severe')

    image_path = '/content/'+img_path
        
    #Using YOLO to detect the damage type
    %cd darknet
    temp = !./darknet detector test /content/custom.data /content/yolov3_custom.cfg /content/yolov3_custom_4000.weights -dont_show </content/test.txt> result.txt

    predicted = load_img('predictions.jpg')
    predicted.save('/content/pred.jpg')
    
    %cd /content/

    return report_pred

In [9]:
def get_response_image(image_path):
    pil_img = Image.open(image_path, mode='r') # reads the PIL image
    byte_arr = io.BytesIO()
    pil_img.save(byte_arr, format='PNG') # convert the PIL image to byte array
    encoded_img = encodebytes(byte_arr.getvalue()).decode('ascii') # encode as base64
    return encoded_img

In [50]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from werkzeug.utils import secure_filename
from flask import send_file
from flask import Flask, redirect, url_for, request, render_template
app = Flask(__name__)
   #starts ngrok when the app is run

@app.route("/", methods=["GET", "POST"])
def home():
    return render_template('app.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
	if request.method == 'POST':
		f = request.files['file']
		filepath = os.path.join('/content/',f.filename)
		f.save(filepath)
	preds = report(f.filename)
	encoded_img = get_response_image('pred.jpg')
	if preds[0] == 'Damaged':
		jsonObject = {"msg":"The vehicle is damaged on the "+preds[1]+" suffering "+preds[2]+" damages. Below is the type of damage detected.","encodedImg":'<img src="data:image/png;base64, '+encoded_img+' alt="Red dot" style="width:400px;height:400px;" />'}
		return json.dumps(jsonObject)
	else:
		jsonObject = {"msg":"Are you sure the vehicle is damaged?. Please check once."}
		return json.dumps(jsonObject)

In [51]:
run_with_ngrok(app)
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7c68c6f3ecf8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Dec/2020 06:03:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2020 06:03:23] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:24] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:24] "GET /favicon.ico HTTP/1.1" 404 -


/content/darknet


...

/content


127.0.0.1 - - [08/Dec/2020 06:03:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2020 06:03:41] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:41] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:42] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:42] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:03:48] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2020 06:04:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2020 06:04:43] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:04:43] "GET /static/js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:04:43] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2020 06:04:43] "GET /static/js/main.js HTTP/1.1" 404 -


/content/darknet


127.0.0.1 - - [08/Dec/2020 06:04:50] "POST /predict HTTP/1.1" 200 -


/content
